In [1]:
#Importing Dependencies
import os
import numpy as np
import gym
import pickle
import pygame
import stable_baselines3

pygame 2.4.0 (SDL 2.26.4, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Creating Enviorment
env = gym.make("Taxi-v3")
env

<TimeLimit<TaxiEnv<Taxi-v3>>>

In [3]:
#Creating State
state = env.reset() #Initially reseting the enviorment to it's original state.
state

387

In [4]:
#Checking For Obeservation
env.observation_space.n

500

Currently We have total 500 observations available in this enviorment

In [5]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Possible actions:

Down: 0

Up: 1

Right: 2

Left: 3

Pickup: 4

Drop-off: 5

In [6]:
n_states = env.observation_space.n
n_actions = env.action_space.n

In [7]:
n_actions

6

In [8]:
#Pre-setting the enviorment
env.env.s = 150

In [9]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



The | in above figures are barriers(walls), By presetting the enviorment to 150 we have changed the pickup location(Y).

In [10]:
env.step(1)

(50, -1, False, {'prob': 1.0})

step() - Updates an environment with actions returning the next agent observation.
The above code gives us the probability of reaching to that state. So the Probability = 1 means the passanger is inside the cab where as if its 0 then the passanger is not in cab. The value 50 are observations.

# How good agent behaves at complete random approach?

In [11]:
state = env.reset()
counter = 0
g = 0
reward = None 

counter is for the directions. Initially it set to 0

g = goal state wether the agent was successful to drop off to correct locations. Initially it set to 0

reward = None : This is boolean variable passing if it's successful in picking up location where passanger and drop at correct location then it will be True or else it will be False.

In [12]:
while reward != 20:
    state, reward, done, info = env.step(env.action_space.sample())
    counter += 1
    g += reward

In the above while loop we are setting the highest reward 20. If it's not equal to the 20 then we'll enter in while loop.
We are going to initialise the state, reward, done and info variables which are present in step function.

The Step Function will perform all the action at each step, So Whenever the transition happens from one state to another state.
At every state you will have reward, when you will reached from old state(St) to new state(St+1) you will get reward.
It can be positive or negative reward.

Done can be True in only 2 possible states, If it is at pick-up location and When if it drop off at the correct location, Else it will be False.

Information is the current location of Taxi.

Whenever the action is done  we are incrementing the counter and also the reward regardless it is +ve or -ve

In [13]:
print("Solved in {} steps with total reward of {}".format(counter,g))

Solved in 1425 steps with total reward of -5400


The agent Solved in 1425 steps with total reward of -5400 which is complete random.
Now we have to minimise the steps and maximise the rewards.

We're getting negative rewards instead of positive. 

So to get max and +ve rewards we will abandon the random approach

# Using Q-learning

In [14]:
#Initialise the Q table
Q = np.zeros([n_states, n_actions])

In [15]:
Q

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# Training agent for 1 episode

In [16]:
#Defining Hyperparameters
episode = 1 #With how many episode the agent learn the enviroment.
G = 0  #Goal State
alpha = 0.618 #Learning rate

In [17]:
for episode in range(1, episode+1):
    done = False
    G,reward = 0,0
    state = env.reset()
    firstState = state
    print("Initial State = {}".format(state))
    while reward != 20:
        action = np.argmax(Q[state])
        new_state, reward, done, info = env.step(action)
        Q[state,action] += alpha * (reward + np.max(Q[new_state]) - Q[state,action])
        G += reward
        state = new_state

Initial State = 434


In [18]:
FinalState = state
FinalState

410

In [19]:
Q

array([[ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [-1.236   , -0.854076, -1.236   , -1.236   , -6.18    , -6.18    ],
       [ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ]])

# Training agent for multiple episodes

In [20]:
episode = 1000
rewardTracker = [] #List of rewards
G = 0
alpha = 0.618

In [22]:
for episode in range(1, episode+1):
    done = False
    G,reward = 0,0
    state = env.reset()
    while done != True:
        action = np.argmax(Q[state])
        new_state, reward, done, info = env.step(action)
        Q[state,action] += alpha * (reward + np.max(Q[new_state]) - Q[state,action])
        G += reward
        state = new_state
    if episode % 100 == 0:
        print("Episode: {} Total reward: {}".format(episode, G))

Episode: 100 Total reward: 12
Episode: 200 Total reward: 10
Episode: 300 Total reward: 11
Episode: 400 Total reward: 11
Episode: 500 Total reward: 8
Episode: 600 Total reward: 7
Episode: 700 Total reward: 9
Episode: 800 Total reward: 8
Episode: 900 Total reward: 12
Episode: 1000 Total reward: 6


# Visualisation

In [23]:
state = env.reset()
done = None

In [24]:
while done != True:
    #We are taking the highest Q value in the action 
    action = np.argmax(Q[state])
    state, reward, done, info = env.step(action)
    env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y|